In [13]:
# Source: https://github.com/jonathanpaulson/AdventOfCode/blob/master/2022/22.py

# Read a file
# Split into grid and instructions
G, instr = open("puzzle.txt").read().split('\n\n')
G = G.split('\n')
instr = instr.strip()

# Define list of directions
D = [(-1, 0), (0, 1), (1, 0), (0, -1)]

# Get rows and columns
R = len(G)
C = len(G[0])

# Add blank spaces for the grid
for r in range(R):
    while len(G[r]) < C:
        G[r] += ' '

# Define the cube
CUBE = C // 3

# Define regions of the cube
REGION = [(0, 1), (0, 2), (1, 1), (2, 1), (2, 0), (3, 0)]

def newCoords(r, c, d, nd):
    '''
    Figure out new coordinates

    Input:
        r (int): current row
        c (int): current column
        d (int): current direction
        nd (int): new direction
    Output:
        (set of 2 ints): new coordinates
    '''

    if d == 0:
        x = c
    elif d == 1:
        x = r
    elif d == 2:
        x = CUBE - 1 - c
    else:
        x = CUBE - 1 - r

    if nd == 0:
        return (CUBE - 1, x)
    elif nd == 1:
        return (x, 0)
    elif nd == 2:
        return (0, CUBE - 1 - x)
    else:
        return (CUBE - 1 - x, CUBE - 1)

def getDest(r, c, d, part):

    # Wrap around for movement
    if part == 1:
        r = (r + D[d][0]) % R
        c = (c + D[d][1]) % C
        while G[r][c]==' ':
            r = (r + D[d][0]) % R
            c =(c + D[d][1]) % C
        return (r, c, d)

    # For part 2, instead of wrap around, treat it as a cube
    # Get region
    region,rr,rc = [(i + 1, r - rr * CUBE, c - cc * CUBE)
                    for i, (rr, cc) in enumerate(REGION)
                    if (rr * CUBE <= r < (rr + 1) * CUBE and
                        cc * CUBE <= c < (cc + 1) * CUBE)][0]

    # 0=up, 1=right,2=down,3=left
    # If I am leaving region R in direction D, 
    # I enter region NR in direction ND
    newRegion, nd = {
        (4, 0): (3, 0), (4, 1): (2, 3), (4, 2): (6, 3), (4, 3): (5, 3),
        (1, 0): (6, 1), (1, 1): (2, 1), (1, 2): (3, 2), (1, 3): (5, 1),
        (3, 0): (1, 0), (3, 1): (2, 0), (3, 2): (4, 2), (3, 3): (5, 2),
        (6, 0): (5, 0), (6, 1): (4, 0), (6, 2): (2, 2), (6, 3): (1, 2),
        (2, 0): (6, 0), (2, 1): (4, 3), (2, 2): (3, 3), (2, 3): (1, 3),
        (5, 0): (3, 1), (5, 1): (4, 1), (5, 2): (6, 2), (5, 3): (1, 1)}[(region, d)]

    # Get new coordinates
    nr, nc = newCoords(rr, rc, d, nd)

    # Region on cube back to global grid
    nr, nc = (REGION[newRegion - 1][0] * CUBE + nr,
              REGION[newRegion - 1][1] * CUBE + nc)
    return (nr, nc, nd)

def solve(part):
    
    # Get starting location
    r = 0
    c = 0
    d = 1
    while G[r][c] != '.':
        c += 1

    i = 0
    while i < len(instr):
        n = 0

        while i < len(instr) and instr[i].isdigit():
            n = n * 10 + int(instr[i])
            i += 1

        # Execute each instruction
        for _ in range(n):

            rr = (r + D[d][0]) % R
            cc = (c + D[d][1]) % C
            if G[rr][cc]==' ':
                (nr, nc, nd) = getDest(r, c, d, part)

                # If hit a wall, stop
                if G[nr][nc]=='#':
                    break
                (r, c, d) = (nr, nc, nd)
                continue

            # If hit a wall, stop
            elif G[rr][cc] == '#':
                break
            else:
                r = rr
                c = cc
        if i==len(instr):
            break
        turn = instr[i]

        # Make left or right turn
        if turn == 'L':
            d = (d + 3) % 4
        elif turn == 'R':
            d = (d + 1) % 4
        i += 1

    DV = {0:3, 1:0, 2:1, 3:2}
    return (r + 1) * 1000 + (c + 1) * 4 + DV[d]

print(solve(1))
print(solve(2))

30552
184106
